In [152]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/items.csv


# **Create a dataframe for every dataset**

In [207]:
mydateparser = lambda x: pd.datetime.strptime(x, "%m/%d/%y")

In [208]:
shop_data = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
sales_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv',parse_dates=['date'], date_parser=mydateparser)
items_data = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
items_category_data = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


ValueError: time data '02.01.2013' does not match format '%m/%d/%y'

# EDA

In [209]:
print("Number of rows in sales_train:",sales_train.shape[0])
print(sales_train.head())
print("Number of rows in shop_data:",shop_data.shape[0])
print(shop_data.head())
print("Number of rows in items_data:",items_data.shape[0])
print(items_data.head())
print("Number of rows :",items_category_data.shape[0])
print(items_category_data.head())
print("Number of rows :",test.shape[0])
print(test.head())

Number of rows in sales_train: 2935849
        date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0 2013-01-02               0       59    22154      999.00           1.0
1 2013-01-03               0       25     2552      899.00           1.0
2 2013-01-05               0       25     2552      899.00          -1.0
3 2013-01-06               0       25     2554     1709.05           1.0
4 2013-01-15               0       25     2555     1099.00           1.0
Number of rows in shop_data: 60
                        shop_name  shop_id
0   !Якутск Орджоникидзе, 56 фран        0
1   !Якутск ТЦ "Центральный" фран        1
2                Адыгея ТЦ "Мега"        2
3  Балашиха ТРК "Октябрь-Киномир"        3
4        Волжский ТЦ "Волга Молл"        4
Number of rows in items_data: 22170
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2 

# # Data Leakage

In [210]:
train = sales_train

In [211]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [214]:
train.date.max()

Timestamp('2015-10-31 00:00:00')

In [215]:
train['date'].dt.year

0          2013
1          2013
2          2013
3          2013
4          2013
           ... 
2935844    2015
2935845    2015
2935846    2015
2935847    2015
2935848    2015
Name: date, Length: 2935849, dtype: int64

In [221]:
part = train[(train['date'].dt.month == 10) & (train['date'].dt.year == 2015)]
part

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2882335,2015-10-23,33,45,13315,649.0,1.0
2882336,2015-10-05,33,45,13880,229.0,1.0
2882337,2015-10-02,33,45,13881,659.0,1.0
2882338,2015-10-12,33,45,13881,659.0,1.0
2882339,2015-10-04,33,45,13923,169.0,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.0,1.0
2935845,2015-10-09,33,25,7460,299.0,1.0
2935846,2015-10-14,33,25,7459,349.0,1.0
2935847,2015-10-22,33,25,7440,299.0,1.0


In [189]:
part['date'].max()

NaT

In [161]:
part['date'].min()

Timestamp('2015-12-01 00:00:00')

In [162]:
part.sort_values(by = 'shop_id')

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2862687,2015-12-09,32,2,1534,675.50,1.0
2725074,2015-12-07,30,2,7793,899.25,2.0
2725064,2015-12-07,30,2,7842,1599.00,1.0
2724802,2015-12-07,30,2,1564,1499.50,1.0
2724763,2015-12-07,30,2,3421,699.00,1.0
...,...,...,...,...,...,...
2701050,2015-12-06,29,59,14227,99.00,1.0
2701283,2015-12-06,29,59,21872,148.00,1.0
2701291,2015-12-06,29,59,21404,169.00,1.0
2898992,2015-12-10,33,59,10200,399.00,1.0


In [163]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [164]:
part.drop(['date', 'date_block_num', 'item_price'], axis = 1 , inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [165]:
part.nunique()

shop_id           51
item_id         4474
item_cnt_day      31
dtype: int64

In [166]:
test.nunique()

ID         214200
shop_id        42
item_id      5100
dtype: int64

In [167]:
part.head()

,shop_id,item_id,item_cnt_day
2323465,27,20611,1.0
2323516,27,19571,1.0
2323629,27,21389,1.0
2323751,27,20949,-1.0
2323786,27,16011,1.0


In [168]:
part.sort_values(['shop_id', 'item_id'], inplace = True)
part.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,shop_id,item_id,item_cnt_day
2530393,2,486,1.0
2724683,2,687,1.0
2724673,2,1288,1.0
2724685,2,1384,1.0
2705136,2,1485,1.0


In [169]:
part.shape

(18252, 3)

In [170]:
part = part.groupby(['shop_id','item_id'], as_index = False).sum()

In [171]:
part.shape

(15486, 3)

In [172]:
test.shape

(214200, 3)

In [173]:
final = pd.merge(part, test, how = 'right', on = ['shop_id','item_id']).fillna(0)
final

,shop_id,item_id,item_cnt_day,ID
0,5,5037,0.0,0
1,5,5320,0.0,1
2,5,5233,0.0,2
3,5,5232,0.0,3
4,5,5268,0.0,4
...,...,...,...,...
214195,45,18454,0.0,214195
214196,45,16188,0.0,214196
214197,45,15757,0.0,214197
214198,45,19648,0.0,214198


In [174]:
a = final.drop(['item_id','shop_id'], axis = 1)

In [175]:
a

,item_cnt_day,ID
0,0.0,0
1,0.0,1
2,0.0,2
3,0.0,3
4,0.0,4
...,...,...
214195,0.0,214195
214196,0.0,214196
214197,0.0,214197
214198,0.0,214198


In [176]:
a = a.rename(columns={'item_cnt_day':'item_cnt_month'})
a = a[['ID','item_cnt_month']]
a

,ID,item_cnt_month
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
214195,214195,0.0
214196,214196,0.0
214197,214197,0.0
214198,214198,0.0


In [177]:
a['item_cnt_month'] = a['item_cnt_month'].clip(lower = 0 ,upper = 20)

In [178]:
a.to_csv('submission.csv', index = False)